# Example: Online Stochastic Bandit Algorithms for Portfolio Rebalancing
In this example, we'll propose a stoachastic multi-armed bandit approach to portfolio rebalancing which a bandit algorithm selectes which assets to include in the portfolio, and an aoptimal Utility Maximization (UM) approach is used to determine the optimal weights for the selected assets. Thus, we are simultaneously addressing the asset selection and weight allocation problems in portfolio management.

> __Learning Objectives__
> 
> By the end of this example, you should be able to:
> Three learning objectives go here.
>

This material will be presented at the upcoming [INFORMS Annual Meeting 2024](https://meetings.informs.org/annual2024/) in Atlanta, GA on October 26-29, 2025.

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [ ]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 